In [1]:
!pip install datasets
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.6 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer,AutoModelForSequenceClassification,DataCollatorWithPadding,TrainingArguments,Trainer
import numpy as np
import torch
from peft import LoraConfig,get_peft_model

In [3]:
dataset=load_dataset('shawhin/imdb-truncated')
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/592 [00:00<?, ?B/s]

(…)-00000-of-00001-5a744bf76a1d84b2.parquet:   0%|          | 0.00/836k [00:00<?, ?B/s]

(…)-00000-of-00001-a3a52fabb70c739f.parquet:   0%|          | 0.00/853k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
})

In [4]:
model_name='distilbert-base-uncased'
tokenizer=AutoTokenizer.from_pretrained(model_name,add_prefix_space=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [5]:
id2label={1:'Positive',0:'Negative'}
label2id={'Positive':1,'Negative':0}

model=AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=2,id2label=id2label,label2id=label2id)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
def tokenize_text(dataset):
  text=dataset['text']

  tokenizer.truncation_side='left'
  tokenized_input=tokenizer(
      text,
      return_tensors='np',
      truncation=True,
      max_length=512
  )
  return tokenized_input

  if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token':['PAD']})
    model.resize_token_embeddings(len(tokenizer))

tokenized_input=dataset.map(tokenize_text,batched=True)
tokenized_input

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [7]:
data_collator=DataCollatorWithPadding(tokenizer=tokenizer)

In [8]:
text_list = ["It was good.", "Not a fan, don't recommed.", "Better than the first one.", "This is not worth watching even once.", "This one is a pass."]

for text in text_list:
  encoded_input=tokenizer.encode(text,return_tensors='pt')
  logits=model(encoded_input).logits
  prediction=torch.argmax(logits)
  print(text,' - ',id2label[prediction.tolist()])

It was good.  -  Positive
Not a fan, don't recommed.  -  Positive
Better than the first one.  -  Positive
This is not worth watching even once.  -  Positive
This one is a pass.  -  Positive


In [29]:
peft_config=LoraConfig(
  task_type='SEQ_CLS',r=4,lora_alpha=32,lora_dropout=0.1,target_modules=['q_lin']
)

In [30]:
model=get_peft_model(model,peft_config)
model.print_trainable_parameters()

trainable params: 628,994 || all params: 67,584,004 || trainable%: 0.9307


In [32]:
training_args=TrainingArguments(
    output_dir="-lora-text-classification",
    learning_rate=0.001,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.1,
    evaluation_strategy='epoch',
    save_strategy="epoch",
    load_best_model_at_end=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [33]:
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
  logits=eval_pred.predictions
  labels=eval_pred.label_ids
  prediction=np.argmax(logits,axis=1)
  accuracy=accuracy_score(labels,prediction)
  return {"Accuracy":accuracy}

In [34]:
trainer=Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    train_dataset=tokenized_input['train'],
    eval_dataset=tokenized_input['validation']
)

<ipython-input-34-2320050a8239>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer=Trainer(


In [35]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.356649,0.879000
2,0.417400,0.520224,0.865000
3,0.417400,0.589487,0.872000
4,0.158800,0.598369,0.880000
5,0.158800,0.647408,0.881000


TrainOutput(global_step=1250, training_loss=0.24366636810302733, metrics={'train_runtime': 218.8418, 'train_samples_per_second': 22.848, 'train_steps_per_second': 5.712, 'total_flos': 556790525519424.0, 'train_loss': 0.24366636810302733, 'epoch': 5.0})

In [49]:
import torch

text = 'this movie was bad'
input_encoded = tokenizer.encode(text, return_tensors='pt')

if torch.cuda.is_available():
  input_encoded = input_encoded.to('cuda')

logits = model(input_encoded).logits
prediction = torch.argmax(logits).item()
print(prediction, '-', id2label[prediction])

0 - Negative
